In [3]:
import os
import re
import sys # Import sys for exit

def synchronize_renumbering(dir1, dir2, prefix="cell_", extension=".png"):
    """
    Renumbers files matching prefix/extension in two directories
    so they are sequential and synchronized based on the union of their numbers.
    Assumes dir1 and dir2 are valid directory paths.
    """
    # --- Get the union of all IDs from both directories for this specific pair ---
    def get_ids(directory):
        ids = set()
        # Compile pattern inside where it's used, or pass it as an argument
        pattern = re.compile(rf"^{re.escape(prefix)}(\d+){re.escape(extension)}$")
        try:
            for filename in os.listdir(directory):
                match = pattern.match(filename)
                if match:
                    try:
                        ids.add(int(match.group(1)))
                    except ValueError:
                        print(f"Warning: Could not parse number from {filename} in {directory}")
                # else: # Optional: Debugging for non-matching files
                #    if filename.startswith(prefix) and filename.endswith(extension):
                #         print(f"Debug: File {filename} in {directory} did not match pattern strictly.")
            return ids
        except FileNotFoundError:
            print(f"Warning: Directory not found while getting IDs: {directory}")
            return set() # Return empty set if dir disappears during run
        except Exception as e:
            print(f"Error reading directory {directory} while getting IDs: {e}")
            return set()

    try:
        # --- Get IDs, sort, and create mapping ---
        all_ids = get_ids(dir1).union(get_ids(dir2))
        if not all_ids:
            print(f"  No files matching pattern found in either directory for pair ({os.path.basename(dir1)}, {os.path.basename(dir2)}). Skipping.")
            return # Nothing to do if no matching files

        sorted_ids = sorted(list(all_ids)) # Sort the unique IDs
        id_mapping = {old_id: new_id for new_id, old_id in enumerate(sorted_ids, start=1)}

        # --- Apply the mapping using temporary files ---
        # Process files based on the sorted order of OLD ids to minimize conflicts
        files_to_rename = []
        for old_id in sorted_ids:
            new_id = id_mapping[old_id]
            if old_id != new_id: # Only need to rename if the ID changes
                 files_to_rename.append((old_id, new_id))

        # Step 1: Rename all necessary files to temporary names
        temp_rename_success = {} # Track which files were successfully temp-renamed { (directory, old_id) : temp_path }
        for old_id, new_id in files_to_rename:
             for directory in [dir1, dir2]:
                 old_filename = f"{prefix}{old_id}{extension}"
                 old_path = os.path.join(directory, old_filename)
                 if os.path.exists(old_path):
                      # Use a more unique temp name if needed, but this often works
                      temp_filename = f"{prefix}{old_id}_TEMP_{new_id}{extension}"
                      temp_path = os.path.join(directory, temp_filename)
                      try:
                          # Avoid renaming if temp file already exists (e.g., interrupted run)
                          if not os.path.exists(temp_path):
                              os.rename(old_path, temp_path)
                              temp_rename_success[(directory, old_id)] = temp_path
                              # print(f"Debug: Renamed {old_path} to {temp_path}") # Verbose Debug
                          else:
                              # If temp exists, assume old one might be gone, add anyway for step 2
                              temp_rename_success[(directory, old_id)] = temp_path
                              print(f"Warning: Temporary file {temp_path} already exists. Assuming previous step completed.")

                      except OSError as e:
                          print(f"  Error renaming {old_filename} to temporary name in {os.path.basename(directory)}: {e}")


        # Step 2: Rename temporary files to their final names
        renamed_count = 0
        for (directory, old_id), temp_path in temp_rename_success.items():
            new_id = id_mapping[old_id] # Get the target new ID again
            new_filename = f"{prefix}{new_id}{extension}"
            new_path = os.path.join(directory, new_filename)

            try:
                 # Important: Check if the FINAL target path somehow exists AND is NOT the temp file itself
                 # This covers edge cases like renaming A->B when B already exists and wasn't part of sync
                 if os.path.exists(new_path) and os.path.abspath(new_path) != os.path.abspath(temp_path):
                      print(f"Warning: Final target file {new_path} already exists. Removing before rename.")
                      os.remove(new_path)

                 os.rename(temp_path, new_path)
                 print(f"  Renamed {os.path.basename(temp_path)} (originally {prefix}{old_id}{extension}) to {new_filename} in {os.path.basename(directory)}")
                 renamed_count += 1
            except OSError as e:
                 print(f"  Error renaming temporary file {os.path.basename(temp_path)} to {new_filename} in {os.path.basename(directory)}: {e}")

        if renamed_count > 0:
             print(f"  Renumbering complete for this pair ({os.path.basename(dir1)}, {os.path.basename(dir2)}).")
        #else:
        #     print(f"  No renumbering needed for this pair.") # Optional message


    except Exception as e:
        # Catch unexpected errors during the main processing of a pair
        print(f"An unexpected error occurred processing pair ({os.path.basename(dir1)}, {os.path.basename(dir2)}): {e}")


# --- Main Execution ---

# Define base directories
base_zdjecia_dir = r"C:\Users\karol\Downloads\dane\niezabudowane\zdjecia"
base_mapy_dir = r"C:\Users\karol\Downloads\dane\niezabudowane\mapy"

# Define file pattern components (allows easy modification)
file_prefix = "cell_"
file_extension = ".png"

print(f"Starting synchronized renumbering for pattern '{file_prefix}*{file_extension}'")
print(f"in corresponding subdirectories of:")
print(f"  Base Dir 1: {base_zdjecia_dir}")
print(f"  Base Dir 2: {base_mapy_dir}")
print("=" * 50)

# --- Basic checks for base directories ---
if not os.path.isdir(base_zdjecia_dir):
    print(f"Error: Base directory 1 not found: {base_zdjecia_dir}")
    sys.exit(1) # Exit the script if base dir is missing

if not os.path.isdir(base_mapy_dir):
    print(f"Error: Base directory 2 not found: {base_mapy_dir}")
    sys.exit(1) # Exit the script if base dir is missing

processed_pairs_count = 0
skipped_non_dir = 0
skipped_missing_pair = 0

# --- Iterate through items in the first base directory ---
try:
    items_in_zdjecia = os.listdir(base_zdjecia_dir)
except OSError as e:
    print(f"Error listing directory {base_zdjecia_dir}: {e}")
    sys.exit(1)

for item_name in items_in_zdjecia:
    # Construct the full path for the item in the first base directory
    path_in_zdjecia = os.path.join(base_zdjecia_dir, item_name)

    # Check if this item is actually a directory
    if os.path.isdir(path_in_zdjecia):
        # Construct the expected corresponding path in the second base directory
        path_in_mapy = os.path.join(base_mapy_dir, item_name)

        # Check if the corresponding directory exists in the second base directory
        if os.path.isdir(path_in_mapy):
            # If both corresponding subdirectories exist, process them
            print(f"Processing pair: '{item_name}'")
            # Pass the prefix and extension to the function
            synchronize_renumbering(path_in_zdjecia, path_in_mapy, prefix=file_prefix, extension=file_extension)
            processed_pairs_count += 1
            print("-" * 30) # Separator for clarity
        else:
            # If the corresponding directory is missing in the second base directory
            print(f"Skipping '{item_name}': Corresponding directory not found in {base_mapy_dir}")
            skipped_missing_pair += 1
            print("-" * 30)
    else:
        # If the item in the first base directory is not a directory (e.g., a file)
        # print(f"Skipping non-directory item: {item_name}") # Optional: uncomment to see skipped files
        skipped_non_dir += 1

# --- Final Summary ---
print("=" * 50)
print("Overall renumbering process finished.")
print(f"Summary:")
print(f"  Processed {processed_pairs_count} pair(s) of corresponding subdirectories.")
if skipped_missing_pair > 0:
    print(f"  Skipped {skipped_missing_pair} item(s) because the corresponding subdirectory was missing in the other base directory.")
if skipped_non_dir > 0:
    print(f"  Skipped {skipped_non_dir} non-directory item(s) found in {base_zdjecia_dir}.")

Starting synchronized renumbering for pattern 'cell_*.png'
in corresponding subdirectories of:
  Base Dir 1: C:\Users\karol\Downloads\dane\niezabudowane\zdjecia
  Base Dir 2: C:\Users\karol\Downloads\dane\niezabudowane\mapy
Processing pair: 'gdansk'
------------------------------
Processing pair: 'warszawa'
  Renamed cell_20_TEMP_18.png (originally cell_20.png) to cell_18.png in warszawa
  Renamed cell_20_TEMP_18.png (originally cell_20.png) to cell_18.png in warszawa
  Renamed cell_21_TEMP_19.png (originally cell_21.png) to cell_19.png in warszawa
  Renamed cell_21_TEMP_19.png (originally cell_21.png) to cell_19.png in warszawa
  Renamed cell_22_TEMP_20.png (originally cell_22.png) to cell_20.png in warszawa
  Renamed cell_22_TEMP_20.png (originally cell_22.png) to cell_20.png in warszawa
  Renamed cell_23_TEMP_21.png (originally cell_23.png) to cell_21.png in warszawa
  Renamed cell_23_TEMP_21.png (originally cell_23.png) to cell_21.png in warszawa
  Renumbering complete for this pai